<a href="https://colab.research.google.com/github/cmplx-xyttmt/mlops/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLOps workflow

We'll be using GCP and Vertex AI.

## Setting up GCP

In [3]:
# Login to gcloud
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=WfoY2S97RwtVeTvTaS28cgpHa2Zojw&prompt=consent&access_type=offline&code_challenge=SBOxZBdviOUgFyHdJajYC1MjtHFOGsjwgY09LWMMNP8&code_challenge_method=S256

Enter authorization code: 4/0AbUR2VOQrCmqfCFZE3He_T7-i1fc39yBkZTRKIqb03GpTs789ymxMvmFoHLTqtkEiAkp_Q

You are now logged in as [iowomugisha@sunbird.ai].
Your current project is [None].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [4]:
# Set the working project
!gcloud config set project sb-gcp-project-01

Updated property [core/project].


In [5]:
# Confirm that the project is set
!gcloud config list project

[core]
project = sb-gcp-project-01

Your active configuration is: [default]


In [ ]:
# (Optional) Enable required services.
!gcloud services enable \
  compute.googleapis.com \
  iam.googleapis.com \
  iamcredentials.googleapis.com \
  monitoring.googleapis.com \
  logging.googleapis.com \
  notebooks.googleapis.com \
  aiplatform.googleapis.com \
  bigquery.googleapis.com \
  artifactregistry.googleapis.com \
  cloudbuild.googleapis.com \
  container.googleapis.com

In [9]:
# create a service account for custom training
import os
os.environ['SERVICE_ACCOUNT_ID'] = 'vertex-custom-training-sunbird'
!echo $SERVICE_ACCOUNT_ID
!gcloud iam service-accounts create $SERVICE_ACCOUNT_ID --description="A custom service account for Vertex custom training." --display-name="Vertex AI Custom Training"

vertex-custom-training-sunbird
Created service account [vertex-custom-training-sunbird].


In [14]:
# Grant service account access to Cloud Storage
PROJECT_ID=!(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]
os.environ['PROJECT_ID'] = PROJECT_ID

!gcloud projects add-iam-policy-binding $PROJECT_ID --member=serviceAccount:$SERVICE_ACCOUNT_ID@$PROJECT_ID.iam.gserviceaccount.com --role="roles/storage.admin"

Updated IAM policy for project [sb-gcp-project-01].
bindings:
- members:
  - serviceAccount:service-379507182035@gcp-sa-aiplatform-cc.iam.gserviceaccount.com
  role: roles/aiplatform.customCodeServiceAgent
- members:
  - serviceAccount:asr-model-srvc@sb-gcp-project-01.iam.gserviceaccount.com
  - serviceAccount:service-379507182035@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - serviceAccount:service-379507182035@gcp-sa-artifactregistry.iam.gserviceaccount.com
  role: roles/artifactregistry.serviceAgent
- members:
  - serviceAccount:379507182035@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - serviceAccount:service-379507182035@gcp-sa-cloudbuild.iam.gserviceaccount.com
  role: roles/cloudbuild.serviceAgent
- members:
  - serviceAccount:service-379507182035@compute-system.iam.gserviceaccount.com
  role: roles/compute.serviceAgent
- members:
  - serviceAccount:service-379507182035@containerregistry.iam.g

In [15]:
!gcloud projects add-iam-policy-binding $PROJECT_ID --member=serviceAccount:$SERVICE_ACCOUNT_ID@$PROJECT_ID.iam.gserviceaccount.com --role="roles/aiplatform.user"

Updated IAM policy for project [sb-gcp-project-01].
bindings:
- members:
  - serviceAccount:service-379507182035@gcp-sa-aiplatform-cc.iam.gserviceaccount.com
  role: roles/aiplatform.customCodeServiceAgent
- members:
  - serviceAccount:asr-model-srvc@sb-gcp-project-01.iam.gserviceaccount.com
  - serviceAccount:service-379507182035@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - serviceAccount:vertex-custom-training-sunbird@sb-gcp-project-01.iam.gserviceaccount.com
  role: roles/aiplatform.user
- members:
  - serviceAccount:service-379507182035@gcp-sa-artifactregistry.iam.gserviceaccount.com
  role: roles/artifactregistry.serviceAgent
- members:
  - serviceAccount:379507182035@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - serviceAccount:service-379507182035@gcp-sa-cloudbuild.iam.gserviceaccount.com
  role: roles/cloudbuild.serviceAgent
- members:
  - serviceAccount:service-379507182035@compute-system.

## Data exploration